# Modèle de Classification detection d'anomalies

https://blog.floydhub.com/introduction-to-anomaly-detection-in-python/

https://www.youtube.com/watch?v=c5V7gSTxS_0

## 1. Librairies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Dataset metrics

In [2]:
# nom de fichier et chemin relatif
filename = 'metrics.csv'
path = '../data/'
# création d'un dataframe à partir du csv de données
df = pd.read_csv(path+filename, index_col=0)
df.head(2)

,status,created_at,cyan_capacity,cyan_remaining,magenta_capacity,magenta_remaining,yellow_capacity,yellow_remaining,black_capacity,black_remaining,machineId,connected_operators,varnishLevelsTargetvolume,varnishLevelsTotalvolume,modules,events
id,,,,,,,,,,,,,,,,
5561325,WARNING,2022-06-07 12:23:52.164000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,"[{""name"": ""JAN"", ""level"": ""Operator""}]",22050.818386,100000,"[{""sn"": """", ""name"": ""Print Engine 1"", ""type"": ...",[]
5513803,IDLE,2022-06-03 08:44:57.996000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,"[{""name"": ""Viktor"", ""level"": ""Operator""}]",22402.256226,100000,"[{""sn"": """", ""name"": ""Print Engine 1"", ""type"": ...",[]


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1164430 entries, 5561325 to 10909319
Data columns (total 16 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   status                     1164430 non-null  object 
 1   created_at                 1164430 non-null  object 
 2   cyan_capacity              0 non-null        float64
 3   cyan_remaining             0 non-null        float64
 4   magenta_capacity           0 non-null        float64
 5   magenta_remaining          0 non-null        float64
 6   yellow_capacity            0 non-null        float64
 7   yellow_remaining           0 non-null        float64
 8   black_capacity             0 non-null        float64
 9   black_remaining            0 non-null        float64
 10  machineId                  1164430 non-null  int64  
 11  connected_operators        1164430 non-null  object 
 12  varnishLevelsTargetvolume  1164430 non-null  float64
 13  varni

In [6]:
df.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df = df.dropna(axis=1)

In [7]:
df.head(2)

,status,created_at,machineId,connected_operators,varnishLevelsTargetvolume,varnishLevelsTotalvolume,modules,events
0,WARNING,2022-06-07 12:23:52.164000,14,"[{""name"": ""JAN"", ""level"": ""Operator""}]",22050.818386,100000,"[{""sn"": """", ""name"": ""Print Engine 1"", ""type"": ...",[]
1,IDLE,2022-06-03 08:44:57.996000,14,"[{""name"": ""Viktor"", ""level"": ""Operator""}]",22402.256226,100000,"[{""sn"": """", ""name"": ""Print Engine 1"", ""type"": ...",[]


In [18]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Audrey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Audrey\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [92]:
str_ = df.modules.loc[0]
words_in_quote = word_tokenize(str_)
words = [word.lower() for word in words_in_quote if word.isalpha() or word.isnumeric()]
print(words)
len(words)
from sklearn.feature_extraction.text import TfidfVectorizer
# Initialiser le vecteur TF-IDF
tf_idf_vector = TfidfVectorizer()
# Vectoriser les mots
word_vector = tf_idf_vector.fit_transform(words)
# Afficher le vecteur résultant
#print(word_vector.nonzero())
word_vector.shape

['sn', 'name', 'print', 'engine', '1', 'type', 'varnish', 'printer', 'counters', 'name', 'varnish', 'counter', 'value', '1965087', 'generation', 'sn', 'name', 'ifoil', 'l', 'type', 'ifoil', 'counters', 'name', 'total', 'pages', 'counter', 'value', '63822', 'name', 'foiled', 'pages', 'counter', 'value', '36211', 'generation', '2']


(36, 18)

In [102]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn import datasets
from numpy import count_nonzero



# representing in CSR form
X_sparse = csr_matrix(word_vector)
print(X_sparse)

# specify the no of output features
tsvd = TruncatedSVD(n_components=10)

# apply the truncatedSVD function
X_sparse_tsvd = tsvd.fit(X_sparse).transform(X_sparse)
print(X_sparse_tsvd)

# shape of the reduced matrix
print(X_sparse_tsvd.shape)


  (0, 13)	1.0
  (1, 9)	1.0
  (2, 11)	1.0
  (3, 5)	1.0
  (5, 15)	1.0
  (6, 17)	1.0
  (7, 12)	1.0
  (8, 4)	1.0
  (9, 9)	1.0
  (10, 17)	1.0
  (11, 3)	1.0
  (12, 16)	1.0
  (13, 0)	1.0
  (14, 7)	1.0
  (15, 13)	1.0
  (16, 9)	1.0
  (17, 8)	1.0
  (19, 15)	1.0
  (20, 8)	1.0
  (21, 4)	1.0
  (22, 9)	1.0
  (23, 14)	1.0
  (24, 10)	1.0
  (25, 3)	1.0
  (26, 16)	1.0
  (27, 2)	1.0
  (28, 9)	1.0
  (29, 6)	1.0
  (30, 10)	1.0
  (31, 3)	1.0
  (32, 16)	1.0
  (33, 1)	1.0
  (34, 7)	1.0
[[ 6.93889390e-17 -1.16382598e-14 -1.48725362e-15 -1.22642254e-01
  -4.90299277e-01  1.74789549e-01 -4.36852210e-01 -2.00724051e-01
   5.54723903e-01 -4.18527781e-01]
 [ 1.00000000e+00  1.66533454e-16  1.41126878e-16  1.08585273e-17
   2.71239778e-17 -2.47071402e-17  1.41546502e-17  5.68726011e-19
  -4.16333634e-17  5.18146022e-17]
 [ 5.03069808e-17  9.02598309e-16 -3.49666961e-18  3.27208064e-16
  -4.25869401e-17 -3.81987492e-15 -4.36238570e-16  2.69671655e-16
   7.65446734e-16 -1.86005501e-16]
 [-1.11022302e-16 -1.41726908e-1

In [42]:
def vectorize_serie(serie):
    tfid_vectorize = TfidfVectorizer()
    words = [word.lower() for word in word_tokenize(serie) if word.isalpha() or word.isnumeric()]
    word_vector = tfid_vectorize.fit_transform(words)
    return word_vector.resize(1,1)

In [43]:
module_df = df.modules.apply(lambda x : vectorize_serie(x))


In [37]:
#module_df = df.modules.apply(lambda x : [word.lower() for word in word_tokenize(x) if word.isalpha() or word.isnumeric()])

In [53]:
module_df.loc[0].get_shape()

(36, 18)

In [103]:
def sparce_vector(serie):
    X_sparse = csr_matrix(serie)
    #print(X_sparse)

    # specify the no of output features
    tsvd = TruncatedSVD(n_components=10)

    # apply the truncatedSVD function
    X_sparse_tsvd = tsvd.fit(X_sparse).transform(X_sparse)
    #print(X_sparse_tsvd)

    # shape of the reduced matrix
    #print(X_sparse_tsvd.shape)

    return X_sparse_tsvd

In [104]:
sparce_module_df = module_df.apply(lambda x : sparce_vector(x))

In [108]:
sparce_module_df.loc[0]

array([[ 3.20692026e-16, -1.39862080e-16, -9.28541275e-17,
        -4.21614222e-01, -7.06408444e-02,  5.90430653e-01,
        -1.85051299e-02, -4.14600244e-01,  3.69906998e-01,
         3.99469615e-01],
       [ 1.00000000e+00, -6.19700807e-15,  2.30476165e-16,
        -8.32667268e-17,  2.69360552e-15, -1.66533454e-16,
         1.11022302e-16, -1.38777878e-16, -2.22044605e-16,
        -2.22044605e-16],
       [ 5.88412856e-17, -1.91860416e-15, -3.27968334e-17,
        -3.56794294e-16,  2.58126853e-15,  2.62021141e-15,
        -6.54369824e-15, -3.27372561e-18,  2.44217171e-16,
        -5.21318035e-16],
       [-1.49294711e-16, -5.75928194e-16, -1.09985792e-16,
        -6.90732718e-17,  1.22124533e-15, -1.00993291e-15,
        -5.88510675e-15, -2.28870348e-16,  5.60651389e-17,
         7.03992366e-16],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.